In [ ]:
import numpy
import cv2 as cv
from keras.layers import Input, Conv2D, Activation, MaxPool2D, Flatten, Dense
from keras.models import Model
from keras.regularizers import l2
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import csv

In [ ]:
def convert_to_one_hot(v, c):
    v = numpy.eye(c)[v.reshape(-1)]
    return v

In [ ]:
X_train = []
Y_train = []
X_test = []
Y_test = []

with open('/content/drive/MyDrive/train_dataset.csv', 'r') as f:
  reader = csv.reader(f)
  for row in reader:
    if len(row) != 0:
      label = row[0]
      image = numpy.array([int(i) for i in row[1:]], dtype='uint8')
      image = image.reshape((28, 28, 3))
      image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
      image = cv.resize(image, (125, 125))
      X_train.append(image)
      Y_train.append(label)

print('len of X_train: ', len(X_train))
print('len of Y_train: ', len(Y_train))

len of X_train:  62400
len of Y_train:  62400


In [ ]:
with open('/content/drive/MyDrive/test_dataset.csv', 'r') as f:
  reader = csv.reader(f)
  for row in reader:
    if len(row) == 2353:
      label = row[0]
      image = numpy.array([int(i) for i in row[1:]], dtype='uint8')
      image = image.reshape((28, 28, 3))
      image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
      image = cv.resize(image, (125, 125))
      X_test.append(image)
      Y_test.append(label)

print('len of X_test: ', len(X_test))
print('len of Y_test: ', len(Y_test))

len of X_test:  14999
len of Y_test:  14999


In [ ]:
X_train = numpy.array(X_train)
X_test = numpy.array(X_test)
Y_train = numpy.array(Y_train)
Y_test = numpy.array(Y_test)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

X_train = numpy.expand_dims(X_train, axis=-1)
Y_train = to_categorical(Y_train, num_classes=26)

X_test = numpy.expand_dims(X_test, axis=-1)
Y_test = to_categorical(Y_test, num_classes=26)

print('shape of X_train: ', X_train.shape)
print('shape of Y_train: ', Y_train.shape)
print('shape of X_test: ', X_test.shape)
print('shape of Y_test: ', Y_test.shape)

(62400, 125, 125)
(62400,)
(14999, 125, 125)
(14999,)
shape of X_train:  (62400, 125, 125, 1)
shape of Y_train:  (62400, 26)
shape of X_test:  (14999, 125, 125, 1)
shape of Y_test:  (14999, 26)


In [ ]:
def hand_gesture_recognition(input_shape=(125, 125, 1), classes=26):
  X_input = Input(shape=input_shape)
  X = Conv2D(32, (3, 3), strides=(1, 1), padding='same', name='conv1', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=l2(0.0001))(X_input)
  X = Conv2D(32, (8, 8), strides=(1, 1), padding='same', name='conv2', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=l2(0.0001))(X)
  X = Activation(activation='relu')(X)
  X = MaxPool2D((2,2), strides=(2, 2))(X)
  X = Conv2D(64, (8, 8), strides=(1, 1), padding='same', name='conv3', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=l2(0.0001))(X)
  X = Conv2D(32, (8, 8), strides=(1, 1), padding='same', name='conv4', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=l2(0.0001))(X)
  X = Conv2D(64, (2, 2), strides=(1, 1), padding='same', name='conv5', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=l2(0.0001))(X)
  X = Activation(activation='relu')(X)
  X = MaxPool2D((2,2), strides=(2, 2))(X)
  X = Conv2D(64, (4, 4), strides=(1, 1), padding='same', name='conv6', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=l2(0.0001))(X)
  X = Conv2D(64, (4, 4), strides=(1, 1), padding='same', name='conv7', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=l2(0.0001))(X)
  X = Conv2D(64, (3, 3), strides=(1, 1), padding='same', name='conv8', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=l2(0.0001))(X)
  X = Conv2D(64, (3, 3), strides=(1, 1), padding='same', name='conv9', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=l2(0.0001))(X)
  X = Activation(activation='relu')(X)
  X = Flatten()(X)
  X = Dense(64, activation='relu', name='fc1', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=l2(0.0001))(X)
  X = Dense(64, activation='relu', name='fc2', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=l2(0.0001))(X)
  X = Dense(classes, activation='softmax', name='fc4', kernel_initializer=glorot_uniform(seed=0))(X)

  cnn_model = Model(inputs=X_input, outputs=X, name='hand_gesture_recognition')
  return cnn_model

In [ ]:
aug = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.15, zoom_range=0.05, horizontal_flip=False, fill_mode='nearest')
hand_gesture_model = hand_gesture_recognition(input_shape=(125, 125, 1), classes=26)
hand_gesture_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
hand_gesture_model.summary()
hand_gesture_model.fit(aug.flow(X_train, Y_train, batch_size=1024), epochs=50, batch_size=1024)
prediction = hand_gesture_model.evaluate(X_test, Y_test)
print('loss on test set: ', prediction[0])
print('accuracy on test set: ', prediction[1])

Model: "hand_gesture_recognition"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 125, 125, 1)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 125, 125, 32)      320       
_________________________________________________________________
conv2 (Conv2D)               (None, 125, 125, 32)      65568     
_________________________________________________________________
activation (Activation)      (None, 125, 125, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 62, 32)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 62, 62, 64)        131136    
_________________________________________________________________
conv4 (Conv2D)               (None, 62, 62

In [ ]:
hand_gesture_model.save('/content/drive/MyDrive/data_augmentation_model_hgr_v3.h5')